In [1]:
%reset -fs

%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
# from IPython.core.display import display, HTML
from IPython.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("<style>.container { width:98% !important; }</style>"))



import re


In [2]:
import en_core_web_sm

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
df = pd.read_csv('eu_last1m_clean.csv')

/var/folders/gc/kgfpjt4x1d5c1thdk2sl_2hr0000gn/T/ipykernel_24877/2991653027.py:1: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('eu_last1m_clean.csv')


In [5]:
df

,id,created_at,text,user_description,user_location
0,923665261894434816,Thu Oct 26 21:38:53 +0000 2017,i welcome this announcement from potus but it...,Serving Illinois’ 17th District. Former report...,"Illinois & Washington, D.C."
1,923665922010820608,Thu Oct 26 21:41:31 +0000 2017,congrats iflymia emiliotgonzalez receiving j ...,"Senior Advisor @Akin_Gump, @UnivMiami Fellow, ...","Miami, Florida"
2,923664565115785216,Thu Oct 26 21:36:07 +0000 2017,phxchamber drenakusari michelle just azs bcamp...,Official Twitter feed of U.S. Senator Kyrsten ...,NaN
3,923664652298485760,Thu Oct 26 21:36:28 +0000 2017,women earn of what men make that means from t...,I work for #CA38 in Congress. 🇺🇸 Serve on the ...,"Norwalk, CA & Washington, DC"
4,923664106942582784,Thu Oct 26 21:34:18 +0000 2017,seanad unanimously passes genderpaygap bill th...,NaN,Limerick City
...,...,...,...,...,...
1780692,1125114865423613952,Sun May 05 19:07:42 +0000 2019,delighted to welcome borisjohnson to clwydwest...,Conservative MP for Clwyd West / Aelod Seneddo...,North Wales and Westminster
1780693,1124728667555467264,Sat May 04 17:33:05 +0000 2019,stopped in at the churchill united church for ...,Former IAFF (2727) Firefighter. Member of Parl...,"Barrie, Ontario"
1780694,1124391243885289472,Fri May 03 19:12:17 +0000 2019,proest,Beste Kamerlid (GeenStijl). Politicus van het ...,Bourgondisch Kreits
1780695,1124682517389426688,Sat May 04 14:29:42 +0000 2019,fantastic to join isha jp and niaz at their gr...,Former IAFF (2727) Firefighter. Member of Parl...,"Barrie, Ontario"


In [6]:
# df_test = df.loc[0:6,:].copy()
# df_test

In [7]:
df['user_description'] = df['user_description'].astype(str)

In [8]:
df['user_description'][2]

'Official Twitter feed of U.S. Senator Kyrsten Sinema, serving Arizona.'

In [9]:
def strip_punc_lower_http(txt):
   
    txt = re.sub(pattern= "[^\w\s]",
            repl="",
            string=txt).lower()
    
    txt = re.sub(pattern= "[0-9]",
            repl="",
            string=txt)
    
    return txt
strip_punc_lower_http('I am, testing #ALEX > than X 10. https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html')

'i am testing alex  than x  httpsscikitlearnorgstablemodulesgeneratedsklearnfeature_extractiontexttfidfvectorizerhtml'

In [10]:
def label_party(row):
    # party = 'party'
    
    if ('conseravtive' in row or 'tory' in row  or 'republican' in row):
        party = 'Conservative'
    elif ('labor' in row or 'liberal' in row or 'democrat' in row):
        party = 'Labor/Liberal'
    else:
        party =  'na'
    return party
    

label_party(df['user_description'][2])

'na'

In [11]:
df['user_description'] = df['user_description'].apply(lambda x: strip_punc_lower_http(x))



In [12]:
df['party'] = df['user_description'].apply(lambda x: label_party(x))

In [13]:
df

,id,created_at,text,user_description,user_location,party
0,923665261894434816,Thu Oct 26 21:38:53 +0000 2017,i welcome this announcement from potus but it...,serving illinois th district former reporter w...,"Illinois & Washington, D.C.",na
1,923665922010820608,Thu Oct 26 21:41:31 +0000 2017,congrats iflymia emiliotgonzalez receiving j ...,senior advisor akin_gump univmiami fellow cong...,"Miami, Florida",na
2,923664565115785216,Thu Oct 26 21:36:07 +0000 2017,phxchamber drenakusari michelle just azs bcamp...,official twitter feed of us senator kyrsten si...,NaN,na
3,923664652298485760,Thu Oct 26 21:36:28 +0000 2017,women earn of what men make that means from t...,i work for ca in congress serve on the waysme...,"Norwalk, CA & Washington, DC",na
4,923664106942582784,Thu Oct 26 21:34:18 +0000 2017,seanad unanimously passes genderpaygap bill th...,nan,Limerick City,na
...,...,...,...,...,...,...
1780692,1125114865423613952,Sun May 05 19:07:42 +0000 2019,delighted to welcome borisjohnson to clwydwest...,conservative mp for clwyd west aelod seneddol...,North Wales and Westminster,na
1780693,1124728667555467264,Sat May 04 17:33:05 +0000 2019,stopped in at the churchill united church for ...,former iaff firefighter member of parliament ...,"Barrie, Ontario",na
1780694,1124391243885289472,Fri May 03 19:12:17 +0000 2019,proest,beste kamerlid geenstijl politicus van het jaa...,Bourgondisch Kreits,na
1780695,1124682517389426688,Sat May 04 14:29:42 +0000 2019,fantastic to join isha jp and niaz at their gr...,former iaff firefighter member of parliament ...,"Barrie, Ontario",na


In [14]:
index_names = df[df['party']== 'na'].index
index_names

Int64Index([      0,       1,       2,       3,       4,       5,       7,
                  8,       9,      10,
            ...
            1780687, 1780688, 1780689, 1780690, 1780691, 1780692, 1780693,
            1780694, 1780695, 1780696],
           dtype='int64', length=1702025)

In [15]:
df.drop(index_names, inplace=True)

In [16]:
df

,id,created_at,text,user_description,user_location,party
6,923665746193997824,Thu Oct 26 21:40:49 +0000 2017,our early years sector profile includes a bre...,political leader for progressive change former...,"New York, USA",Labor/Liberal
15,923667345079525376,Thu Oct 26 21:47:10 +0000 2017,finally we get to see mike_fabricant on celebr...,socially liberal mp for lichfield believes in ...,"Lichfield in Staffordshire, UK",Labor/Liberal
29,923670582775951360,Thu Oct 26 22:00:02 +0000 2017,european council orders bottles of champagne ...,socially liberal mp for lichfield believes in ...,"Lichfield in Staffordshire, UK",Labor/Liberal
32,923671599789871104,Thu Oct 26 22:04:05 +0000 2017,million households deduct state amp local tax...,representing ca member of transportdems edlabo...,"Concord, CA",Labor/Liberal
43,923669465530818560,Thu Oct 26 21:55:36 +0000 2017,leninology the sequel yay,psychotherapist mother social democrat former...,Dublin,Labor/Liberal
...,...,...,...,...,...,...
1780488,1102608730196242432,Mon Mar 04 16:36:21 +0000 2019,the transport committee in the european parlia...,bavarian socialdemocrat sd mep committee trans...,Brüssel/Amberg,Labor/Liberal
1780518,1108012184833933312,Tue Mar 19 14:27:45 +0000 2019,do you remember the romanian transport associa...,bavarian socialdemocrat sd mep committee trans...,Brüssel/Amberg,Labor/Liberal
1780524,1109762810928685056,Sun Mar 24 10:24:07 +0000 2019,we are one with these words he pm of new zeala...,bavarian socialdemocrat sd mep committee trans...,Brüssel/Amberg,Labor/Liberal
1780575,1114252621257289728,Fri Apr 05 19:45:01 +0000 2019,i am very pleased to see that the government p...,députée fédérale member of parliament parti ...,"Brossard, Quebec, Canada",Labor/Liberal


In [17]:
print("Document Count")
print(df.groupby('party')['text'].count())
print("Word Count")
df.groupby('party').apply(lambda x: x.text.apply(lambda x: len(x.split())).sum())
df['parsed'] = df.text.apply(nlp)

Document Count
party
Conservative     19855
Labor/Liberal    58817
Name: text, dtype: int64
Word Count


In [18]:
corpus = st.CorpusFromParsedDocuments(df, category_col='party', parsed_col='parsed').build()

In [19]:
# from IPython.display import IFrame
# from IPython.display import display, HTML

In [20]:
corpus.get_term_freq_df()

,Labor/Liberal freq,Conservative freq
term,,
our,10588,4075
early,337,66
years,1665,634
sector,197,74
profile,18,12
...,...,...
canadas environmentalists,1,0
environmentalists feminists,1,0
feminists must,1,0


In [21]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['Labor/Liberal_precision'] = term_freq_df['Labor/Liberal freq'] * 1./(term_freq_df['Labor/Liberal freq'] + term_freq_df['Conservative freq'])
term_freq_df['Labor/Liberal_freq_pct'] = term_freq_df['Labor/Liberal freq'] * 1./term_freq_df['Labor/Liberal freq'].sum()
term_freq_df['Labor/Liberal_hmean'] = term_freq_df.apply(lambda x: (hmean([x['Labor/Liberal_precision'], x['Labor/Liberal_freq_pct']])
                                                                   if x['Labor/Liberal_precision'] > 0 and x['Labor/Liberal_freq_pct'] > 0 
                                                                   else 0), axis=1)                                                        
term_freq_df.sort_values(by='Labor/Liberal_hmean', ascending=False).iloc[:10]

,Labor/Liberal freq,Conservative freq,Labor/Liberal_precision,Labor/Liberal_freq_pct,Labor/Liberal_hmean
term,,,,,
the,64035,20981,0.753211,0.023523,0.045621
to,52069,17424,0.749270,0.019127,0.037302
and,30785,9737,0.759711,0.011309,0.022285
of,28265,9501,0.748425,0.010383,0.020482
in,25100,9254,0.730628,0.009220,0.018211
a,23660,7592,0.757072,0.008691,0.017185
for,22487,7465,0.750768,0.008260,0.016341
is,14518,4473,0.764467,0.005333,0.010592
on,13677,5148,0.726534,0.005024,0.009979


In [22]:
def normcdf(x):
    return norm.cdf(x, x.mean(), x.std())
term_freq_df['Labor/Liberal_precision_normcdf'] = normcdf(term_freq_df['Labor/Liberal_precision'])
term_freq_df['Labor/Liberal_freq_pct_normcdf'] = normcdf(term_freq_df['Labor/Liberal_freq_pct'])
term_freq_df['Labor/Liberal_scaled_f_score'] = hmean([term_freq_df['Labor/Liberal_precision_normcdf'], term_freq_df['Labor/Liberal_freq_pct_normcdf']])
term_freq_df.sort_values(by='Labor/Liberal_scaled_f_score', ascending=False).iloc[:10]

,Labor/Liberal freq,Conservative freq,Labor/Liberal_precision,Labor/Liberal_freq_pct,Labor/Liberal_hmean,Labor/Liberal_precision_normcdf,Labor/Liberal_freq_pct_normcdf,Labor/Liberal_scaled_f_score
term,,,,,,,,
politas,1315,0,1.000000,0.000483,0.000966,0.727248,1.000000,0.842089
auspol politas,726,0,1.000000,0.000267,0.000533,0.727248,1.000000,0.842089
dartmouth,518,0,1.000000,0.000190,0.000380,0.727248,0.999910,0.842057
turnbull,928,2,0.997849,0.000341,0.000682,0.725444,1.000000,0.840878
politas auspol,358,0,1.000000,0.000132,0.000263,0.727248,0.995065,0.840334
trumpshutdown,388,1,0.997429,0.000143,0.000285,0.725091,0.997437,0.839734
dcyapress,336,0,1.000000,0.000123,0.000247,0.727248,0.992239,0.839325
forthepeople,323,0,1.000000,0.000119,0.000237,0.727248,0.989970,0.838512
morrison,523,4,0.992410,0.000192,0.000384,0.720854,0.999923,0.837759


In [23]:
term_freq_df['Labor/Liberal_corner_score'] = corpus.get_corner_scores('Labor/Liberal')
term_freq_df.sort_values(by='Labor/Liberal_corner_score', ascending=False).iloc[:10]

,Labor/Liberal freq,Conservative freq,Labor/Liberal_precision,Labor/Liberal_freq_pct,Labor/Liberal_hmean,Labor/Liberal_precision_normcdf,Labor/Liberal_freq_pct_normcdf,Labor/Liberal_scaled_f_score,Labor/Liberal_corner_score
term,,,,,,,,,
politas,1315,0,1.0,0.000483,0.000966,0.727248,1.000000,0.842089,0.879553
auspol politas,726,0,1.0,0.000267,0.000533,0.727248,1.000000,0.842089,0.879553
dartmouth,518,0,1.0,0.000190,0.000380,0.727248,0.999910,0.842057,0.879552
politas auspol,358,0,1.0,0.000132,0.000263,0.727248,0.995065,0.840334,0.879552
dcyapress,336,0,1.0,0.000123,0.000247,0.727248,0.992239,0.839325,0.879552
forthepeople,323,0,1.0,0.000119,0.000237,0.727248,0.989970,0.838512,0.879552
the turnbull,309,0,1.0,0.000114,0.000227,0.727248,0.986899,0.837408,0.879552
media release,308,0,1.0,0.000113,0.000226,0.727248,0.986651,0.837319,0.879552
merrionstreet,301,0,1.0,0.000111,0.000221,0.727248,0.984802,0.836652,0.879552


In [24]:
term_freq_df = corpus.get_term_freq_df()
term_freq_df['Conservative Score'] = corpus.get_scaled_f_scores('Conservative')
term_freq_df['Labor/Liberal Score'] = corpus.get_scaled_f_scores('Labor/Liberal')
print("Top 10 Labor/Liberal terms")
pprint(list(term_freq_df.sort_values(by='Labor/Liberal Score', ascending=False).index[:10]))
print("Top 10 Conservative terms")
pprint(list(term_freq_df.sort_values(by='Conservative Score', ascending=False).index[:10]))

Top 10 Labor/Liberal terms
['politas',
 'auspol politas',
 'dartmouth',
 'turnbull',
 'morrison',
 'australian',
 'trumpshutdown',
 'wa',
 'labors',
 'australians']
Top 10 Conservative terms
['north country',
 'derry',
 'swindon',
 'm_andersonsf',
 'twill',
 'ny',
 'féin',
 'sinn féin',
 'trolleys',
 'taxreform']


In [25]:
html = produce_scattertext_explorer(corpus,
                                    category='Labor/Liberal',
                                    category_name='Labor/Liberal',
                                    not_category_name='Conservative',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.percentile,
                                    metadata=df['user_location'])
file_name = 'texts.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [ ]:
html = produce_scattertext_explorer(corpus,
                                    category='Labor/Liberal',
                                    category_name='Labor/Liberal',
                                    not_category_name='Conservative',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    
                                    metadata=df['user_location'],
                                    term_significance = st.LogOddsRatioUninformativeDirichletPrior())
file_name = 'texts2.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)